In [68]:
using Distributions
using Printf


function climb!(t, q, A, c, λ, σ; maxit)
    lt = copy(t)

    n = size(t, 1)
    v = rand(Binomial(1, .01), n)

    for _ in 1:maxit
        q .= ifelse.(A' * t - λ - σ + 2 * σ .* t .> 0, 1.0, 0.0)
        t .= ifelse.(A * q + v + λ - σ + 2 * σ .* q .> 0, 1.0, 0.0)

        if all(lt .== t)
            break
        end

        lt .= t
    end
end


function disturbed(A, μ)
    D = randn(size(A))
    A + μ * (D - D')
end


function posneg!(A)
    n = size(A, 1)

    for _ in 1:5
        i = 1 + mod(rand(Int), n)
        for j in 1:n
            A[j, i] += A[i, j]
            A[i, j] = 0
        end
    end
end


function ubqp(A, c; lr=1.0, μ=0.0, decay=.999, lrdecay=sqrt(.999), mdecay=sqrt(.999), maxit)
    W = eltype(A)
    n, n = size(A)

    t = zeros(n)
    q = zeros(n)
    λ = zeros(n)
    σ = zeros(n)

    tstar = copy(t)
    best = -Inf

    M = disturbed(A, μ)

    for it in 1:maxit
        posneg!(M)
        climb!(t, q, M, c, λ, σ, maxit=100)

        ∇ = q - t
        norm = sum(abs.(∇))

        if norm <= zero(W)
            obj = t' * A * q
            if obj > best
                tstar, best = copy(t), obj
                @printf("best = %5.3f\tlr = %5.3f\tit = %d\n",
                    best, lr, it)
                flush(stdout)
            end

            σ *= decay
            λ *= decay
            lr *= lrdecay
            μ *= mdecay

            M = disturbed(A, μ)
        end

        σ += lr * abs.(∇)
        λ += lr * ∇
    end

    tstar
end

ubqp (generic function with 1 method)

In [69]:
using DelimitedFiles: readdlm


function instance(path)
    open(path) do io
        readline(io)
        readline(io)
        readline(io)

        n, m = split(readline(io))
        n, m = parse.(Int, (n, m))

        if n * n > 1_000_000
            @show n m
            throw(ErrorException("wee bit too large"))
        end

        A = zeros(n, n)
        while !eof(io)
            a, b, w = split(readline(io))
            a, b, w = parse.([Int, Int, Float64], [a, b, w])
            A[a, b] = w
        end

        A
    end
end

instance (generic function with 1 method)